In [4]:
import numpy as np
import cv2
import time
from scipy.stats import expon
from scipy import ndimage
import sys
import os 
import matplotlib.pyplot as plt
from scipy.ndimage import uniform_filter
from skimage.registration import optical_flow_tvl1
from skimage.transform import warp
import subprocess
from function import replace_zeros_with_median,dtof_hist_with_img,create_histogram_from_binary_nods,center_of_mass_test

In [5]:
# Parameters to simulate the data
PATH_TO_HVSR = "/home/ar432/DVSR" # PATH to Super-resolution network download from: https://github.com/facebookresearch/DVSR
name_path = 'TestgithubMiddlebury' # PATH TO SAVE
root_path = '/home/ar432/DVSR/data/'
# Choose ppp and SBR levels 
coeffProba= 0.4         # probability of detection: coeffProba = [1.3,9,40,150, 650] corresponds to ppp = [64,16,4,1,0.25] 
SBR = 0.001             # 1 is pure noise, 0 is pure signal: coeffProba = [0.8, 0.5,0.2,0.05,0.015,0.004,0.001] corresponds to SBR = [0.25,1,4,16,64,256, 1024] 

# Choose amount of movement
SpeedX = 0.2            # displacement X (pixels per binary frame)
SpeedY = 0.2            # displacement Y (pixels per binary frame)
SpeedZ = 0.2            # displacement in Z (time bin per binary frame)

# Choose amount of frames (N_frame must be > 6)
N_frame = 12            # number of RGB frames 
M = 100                 # number of binary frames between two RGB frames
Tbin = M*N_frame        # total number of binary frames 

# Choose maximum number of iterations of the plug and play
N_iterations = 10        # number of iterations of the plug and play method

num_bins = 100          # number of bins in the histogram
SigIRF = 0.7            # STD of the Gaussian IRF
ds_factor = 16          # downsampling factor (fixed by HVSR)




In [6]:
# Load Middlebury disparity and RGB images
disp1 = cv2.imread('./middlebury/disp1.png', cv2.IMREAD_GRAYSCALE)  #disparity map
disparity_map = replace_zeros_with_median(disp1.astype(np.float64)).astype(np.float64) #filter out zeros
disparity_map = np.where(disparity_map == 0, 0.1, disparity_map)

focal_length = 3740.0 # Focal length in pixels
baseline = 160.0/10  
Depth = (focal_length * baseline) / disparity_map #Convert disparity into depth
min_d, max_d = np.amin(Depth), np.amax(Depth)
D = (Depth-min_d) / (max_d - min_d)*100 #normalise between 0 and 100

view1 = cv2.imread('./middlebury/view1.png')  #RGB map
I = view1[:, :, 0].astype(np.float64)
Iref = I.copy() # reference RGB
row, col = I.shape # size of image
N = row * col # number of pixels


In [7]:
# Initialize arrays for storing depth, reference, and reflectivity images
data_image = np.zeros((row * col, Tbin))    # Depth data for each time frame
Ref_image = np.zeros((row, col, Tbin))      # Reference depth data for each time frame
Shifted_Refl = np.zeros((row, col, Tbin))   # Reflectivity data for each time frame
Tind = np.ones(N, dtype=bool)

# Loop over each time frame (binary frames)
for t in range(Tbin): 
    # Shift images in X, Y, Z directions
    Rotation_M = cv2.getRotationMatrix2D((col / 2, row / 2), 0, 1)
    Rotation_M[0, 2] += SpeedX * t
    Rotation_M[1, 2] += SpeedY * t

    # Apply affine transformation for depth and reflectivity images
    Dt = cv2.warpAffine(20 + D + SpeedZ * t, Rotation_M, (col, row), borderValue=0)
    It = cv2.warpAffine(20 + I + SpeedZ * t, Rotation_M, (col, row), borderValue=0)
    Igt = cv2.warpAffine(20 + I, Rotation_M, (col, row), borderValue=0)
    
    # Add noise to depth image based on coeffproba value
    MeanD = Dt + np.sqrt(SigIRF) * np.random.randn(row, col)
    Proba = coeffProba * It / np.max(I)
    Tind = (Proba.flatten() > 0) & (np.round(expon.rvs(scale=Proba.flatten())) == 0)

    # Store data
    data_image[Tind, t] = MeanD.flatten()[Tind]
    Shifted_Refl[:, :, t] = Igt
    Ref_image[:, :, t] = Dt

# Define masks based on the Signal-to-Background Ratio (SBR)
mask = np.random.rand(row * col, Tbin) > SBR
mask_opposite = np.random.rand(row * col, Tbin) <= SBR
random_value = num_bins * np.random.rand(row * col, Tbin)
data_image[data_image > 0] = data_image[data_image > 0] * mask[data_image > 0] + random_value[data_image > 0] * mask_opposite[data_image > 0]

# Calculate the actual SBR ratio
correct_pixels = np.sum(mask[data_image > 0])
bad_pixels = np.sum(mask_opposite[data_image > 0])
print(f'Real SBR is {correct_pixels/bad_pixels}')

# Normalise
data_image = data_image.reshape((row, col, Tbin))
max_val = 120+100*12*SpeedZ
data_image[data_image> max_val] = max_val
data_image = data_image / max_val
Ref_image = Ref_image / max_val

# Crop and downsample the images
val_decalage = 110
data_small = data_image[75-7-16*2+val_decalage:75-7+192+val_decalage+16*2,100+val_decalage-16*2:100+192+val_decalage+16*2,:]
rgb_small = Shifted_Refl[75-7-16*2+val_decalage:75-7+192+val_decalage+16*2,100+val_decalage-16*2:100+192+val_decalage+16*2,:]
ref_small = Ref_image[75-7-16*2+val_decalage:75-7+192+val_decalage+16*2,100+val_decalage-16*2:100+192+val_decalage+16*2,:]
ref_big = ndimage.zoom(ref_small, [4,4,1], order=0)
rgb_big = ndimage.zoom(rgb_small, [4,4,1], order=0)
ds_data = ndimage.zoom(data_small, [1/4,1/4,1], order=0)


# Create paths 
path = f'{root_path}{name_path}'
path_save_depth = os.path.join(path, 'depth')
path_save_depth_GT = os.path.join(path, 'GT_depth')
path_save_histogram = os.path.join(path, 'histogram')
path_save_color = os.path.join(path, 'color')
path_save_histogram_init = os.path.join(path, 'histogram_init')
path_save_depth_xy = os.path.join(path, 'depth_xy')

path_mis = f'{root_path}{name_path}_misaligned'
path_save_depth_mis = os.path.join(path_mis, 'depth')
path_save_histogram_mis = os.path.join(path_mis, 'histogram')
path_save_color_mis = os.path.join(path_mis, 'color')
path_save_histogram_init_mis = os.path.join(path_mis, 'histogram_init')

paths=[path, path_save_depth, path_save_histogram, path_save_color, path_save_histogram_init,path_save_depth_xy,path_save_depth_GT,path_mis,path_save_depth_mis, path_save_histogram_mis, path_save_color_mis, path_save_histogram_init_mis]

for path_index in paths:
    os.makedirs(path_index, exist_ok=True)

    
index = 0 
for frame in range(200,1001,100):
    print(index)
    time1 = time.time()
    # Extract a set of M low-resolution binary frames and upsample to high resolution
    HR_depth = ds_data[:,:,frame:frame+M] # M low resolution binary frames
    HR_depth = ndimage.zoom(HR_depth,[4,4,1], order=0) 

    # Get high-resolution RGB images at the start and end of the frame set
    color_start = rgb_small[:,:,frame]/255 # High resolution RGB at time t 
    Nx, Ny = color_start.shape[0], color_start.shape[1]
    color_end = rgb_small[:,:,frame+M]/255 # High resolution RGB at time t+1

    # Calculate optical flow in X/Y between the start and end RGB frames
    v, u = optical_flow_tvl1(color_end, color_start) # Flow between RGB images
    row_coords, col_coords = np.meshgrid(np.arange(Nx), np.arange(Ny),indexing='ij')
    

    # Align low-resolution depth frames along the X/Y-axis  using flow vectors
    factor = [1-i/(M-1) for i in range(M)] #  alignment factors
    xy_HR_depth = np.empty((Nx, Ny, M))
    mis_HR_depth = np.empty((Nx, Ny, M))
    for i in range(M):
        image = HR_depth[:,:,i]
        alignment_factor = factor[i]
        new_image = warp(image, np.array([row_coords + alignment_factor*v, col_coords + alignment_factor*u]),mode='edge', order=0)
        xy_HR_depth[:,:,i] = new_image
        mis_HR_depth[:,:,i] = image
    time2 = time.time()
    
    # Align depth frames along the Z-axis 
    # Get depth maps at the start and end of the sequence
    number_frames = 5
    depth_maps_start = np.round(xy_HR_depth[:,:,:number_frames]*num_bins)
    depth_maps_end = np.round(xy_HR_depth[:,:,-number_frames:]*num_bins)

    # Create histograms from the binary depth maps
    histogram1 = create_histogram_from_binary_nods(depth_maps_start,num_bins)
    histogram_avg1 = uniform_filter(histogram1, size=(ds_factor*3, ds_factor*3,1), mode='reflect')
    
    histogram2 = create_histogram_from_binary_nods(depth_maps_end,num_bins)
    histogram_avg2 = uniform_filter(histogram2, size=(ds_factor*3, ds_factor*3,1), mode='reflect')

    # Calculate depth displacement in the Z-axis
    depth1 = np.squeeze(np.argmax(histogram_avg1, axis =2 ))/num_bins
    depth2 = np.argmax(histogram_avg2, axis = 2).squeeze()/num_bins
    dz = depth2 - depth1 #  displacement in z 
    dz = ndimage.median_filter(dz, size=16*2)

    # Adjust depth frames based on calculated Z displacement
    xy_HR_depth_new = xy_HR_depth[:,:,:]
    alignment_factors = 1 - np.arange(M) / (M - 1)
    alignment_factors = alignment_factors.reshape((1, 1, M))
    non_zero_mask = xy_HR_depth_new != 0
    dz_broadcasted = np.expand_dims(dz, axis=-1)
    xyz_HR_depth = np.where(non_zero_mask, xy_HR_depth_new + alignment_factors * dz_broadcasted, 0)
    

    # Convert adjusted depth to histogramsM
    xyz_HR_depth = xyz_HR_depth*num_bins
    Nx, Ny = xyz_HR_depth.shape[0], xyz_HR_depth.shape[1]  
    num_maps = 100
    xyz_HR_depth[xyz_HR_depth>num_bins-1] = num_bins-1
    pixel_map = xyz_HR_depth.astype(int)

    IRF_matrix = np.zeros((num_bins, num_bins))
    np.fill_diagonal(IRF_matrix, 1)
    histograms = IRF_matrix[pixel_map.ravel()].reshape(pixel_map.shape[0], pixel_map.shape[1], num_maps, num_bins)
    histograms[pixel_map == 0] = 0


    # Create histograms for misaligned depth (before aligning in X/Y/Z)
    mis_HR_depth = mis_HR_depth*100
    Nx, Ny = mis_HR_depth.shape[0], mis_HR_depth.shape[1]  
    num_maps = 100
    mis_HR_depth[mis_HR_depth>num_bins-1] = num_bins-1
    pixel_map = mis_HR_depth.astype(int)
    IRF_matrix = np.zeros((num_bins, num_bins))
    np.fill_diagonal(IRF_matrix, 1)
    histograms_mis = IRF_matrix[pixel_map.ravel()].reshape(Nx, Ny, num_maps, num_bins)
    histograms_mis[pixel_map == 0] = 0

    # Combine histograms to generate depth
    combined_histograms = histograms.sum(axis=2)
    combined_histograms_mis = histograms_mis.sum(axis=2)

    # Compute final depth from histograms
    depth = center_of_mass_test(combined_histograms)/combined_histograms.shape[2]
    depth = ndimage.zoom(depth, [4,4], order=0)
    depth_mis = center_of_mass_test(combined_histograms_mis)/combined_histograms_mis.shape[2]*10
    depth_mis = ndimage.zoom(depth_mis, [4,4], order=0)

    # Generate low-resolution histogram from high-resolution depths
    color_end = rgb_big[:,:,frame+M]/255
    histogram= dtof_hist_with_img(depth, color_end*255,num_bins)
    histogram_mis= dtof_hist_with_img(depth_mis/10, color_end*255,num_bins)

    # Save ground truth depth and other data for comparison
    GT_depth = ref_big[:,:,frame +M]  # reference depth

    # Save all the generated data for each frame index
    np.save(os.path.join(path, 'GT_depth', f"{index:06d}"+'.npy'), GT_depth)        #ground truth depth
    np.save(os.path.join(path, 'depth_xy', f"{index:06d}"+'.npy'), xy_HR_depth)     #array of M binary frames aligned in XY
    np.save(os.path.join(path, 'depth', f"{index:06d}"+'.npy'), depth)              #LR depth based on aligned histogram
    np.save(os.path.join(path, 'histogram', f"{index:06d}"+'.npy'), histogram)      #LR aligned histogram
    cv2.imwrite(os.path.join(path, 'color', f"{index:06d}"+'.png'),  color_end*255) #HR RGB
    np.save(os.path.join(path, 'histogram_init', f"{index:06d}"+'.npy'), histogram) #LR aligned histogram

    # Save misaligned depth and histogram data for comparison
    np.save(os.path.join(path_mis, 'depth', f"{index:06d}"+'.npy'), depth)                  #LR depth based on misaligned histogram
    np.save(os.path.join(path_mis, 'histogram', f"{index:06d}"+'.npy'), histogram_mis)      #LR misaligned histogram
    cv2.imwrite(os.path.join(path_mis, 'color', f"{index:06d}"+'.png'),  color_end*255)     #HR RGB
    np.save(os.path.join(path_mis, 'histogram_init', f"{index:06d}"+'.npy'), histogram_mis) #LR misaligned histogram

    index = index +1


Real SBR is 996.1670760828582
0
1
2
3
4
5
6
7
8
